In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
steepness = 50    # steepness factor
num_blocks = 10   # number of expression blocks
num_data = 9100   # number of data
test_size = 100   # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## Exact Solver

In [6]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=60)

In [22]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    try:
        xval, objval = model.solve("scip")
        # eval
        params.append(list(p)+list(a))
        sols.append(list(list(xval.values())[0].values()))
        objvals.append(objval)
        conviols.append(sum(model.cal_violation()))
    except:
        params.append(list(p)+list(a))
        sols.append(None)
        objvals.append(None)
        conviols.append(None)
    tock = time.time()
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
df.to_csv("result/rb_exact_50-10.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
print("Number of None values: ", df["Sol"].isna().sum())

100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [1:40:13<00:00, 60.14s/it]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean    63.582075               0.0     60.138493
std     25.764417               0.0      0.341542
min     32.576882               0.0     59.440404
25%     43.602677               0.0     59.931598
50%     56.839336               0.0     60.060533
75%     77.274428               0.0     60.293208
max    143.750384               0.0     61.081650
Number of infeasible solution: 0
Number of None values:  0


## Heuristic - Round

In [23]:
from src.heuristic import naive_round

In [24]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_heur_rnd_50-10.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:49<00:00,  1.09s/it]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean    52.032736          0.305178      1.086002
std     12.989899          0.485807      0.205181
min     24.551208          0.000000      0.794311
25%     43.351718          0.000000      0.913846
50%     50.916983          0.000000      1.027520
75%     57.789929          0.515183      1.164178
max    104.705610          1.722386      1.892454
Number of infeasible solution: 42


## Heuristic - N1

In [25]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [27]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model_heur.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    try:
        xval, objval = model_heur.solve("scip")
        # eval
        params.append(list(p)+list(a))
        sols.append(list(list(xval.values())[0].values()))
        objvals.append(objval)
        conviols.append(sum(model_heur.cal_violation()))
    except:
        params.append(list(p)+list(a))
        sols.append(None)
        objvals.append(None)
        conviols.append(None)
    tock = time.time()
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_heur_n1_50-10.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:16<00:00,  6.04it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean    88.293410               0.0      0.164690
std     35.390073               0.0      0.051742
min     34.826312               0.0      0.120786
25%     59.420381               0.0      0.126149
50%     78.585167               0.0      0.140700
75%    114.883516               0.0      0.170366
max    180.932596               0.0      0.343926
Number of infeasible solution: 0


## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 16            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 2830.30
Epoch 1, Validation Loss: 647.80
Epoch 2, Validation Loss: 287.80
Epoch 3, Validation Loss: 210.76
Epoch 4, Validation Loss: 176.59
Epoch 5, Validation Loss: 165.01
Epoch 6, Validation Loss: 151.86
Epoch 7, Validation Loss: 144.81
Epoch 8, Validation Loss: 122.88
Epoch 9, Validation Loss: 115.69
Epoch 10, Validation Loss: 112.59
Epoch 11, Validation Loss: 105.83
Epoch 12, Validation Loss: 95.66
Epoch 13, Validation Loss: 90.68
Epoch 14, Validation Loss: 88.52
Epoch 15, Validation Loss: 84.22
Epoch 16, Validation Loss: 85.67
Epoch 17, Validation Loss: 79.71
Epoch 18, Validation Loss: 82.23
Epoch 19, Validation Loss: 81.38
Epoch 20, Validation Loss: 77.85
Epoch 21, Validation Loss: 79.81
Epoch 22, Validation Loss: 74.15
Epoch 23, Validation Loss: 71.94
Epoch 24, Validation Loss: 73.78
Epoch 25, Validation Loss: 76.82
Epoch 26, Validation Loss: 82.25
Epoch 27, Validation Loss: 75.54
Epoch 28, Validation Loss: 80.97
Epoch 29, Validation Loss: 71.97
Epoch 3

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_lr_50-10.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 175.72it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean    56.988932          0.000046      0.003338
std     25.866054          0.000464      0.001750
min     16.703403          0.000000      0.001862
25%     39.617259          0.000000      0.002510
50%     50.957301          0.000000      0.003008
75%     66.363043          0.000000      0.003986
max    145.902156          0.004638      0.018153
Number of infeasible solution: 1


## Learnable Threshold

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 16            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                          int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 2290.79
Epoch 1, Validation Loss: 493.74
Epoch 2, Validation Loss: 158.32
Epoch 3, Validation Loss: 136.46
Epoch 4, Validation Loss: 125.15
Epoch 5, Validation Loss: 114.52
Epoch 6, Validation Loss: 101.48
Epoch 7, Validation Loss: 97.03
Epoch 8, Validation Loss: 108.27
Epoch 9, Validation Loss: 81.02
Epoch 10, Validation Loss: 75.33
Epoch 11, Validation Loss: 68.00
Epoch 12, Validation Loss: 65.22
Epoch 13, Validation Loss: 60.06
Epoch 14, Validation Loss: 61.57
Epoch 15, Validation Loss: 66.89
Epoch 16, Validation Loss: 71.29
Epoch 17, Validation Loss: 75.21
Epoch 18, Validation Loss: 66.41
Epoch 19, Validation Loss: 74.29
Epoch 20, Validation Loss: 66.65
Epoch 21, Validation Loss: 70.01
Epoch 22, Validation Loss: 67.53
Epoch 23, Validation Loss: 68.88
Epoch 24, Validation Loss: 73.87
Epoch 25, Validation Loss: 72.85
Epoch 26, Validation Loss: 67.64
Epoch 27, Validation Loss: 66.63
Epoch 28, Validation Loss: 76.31
Epoch 29, Validation Loss: 69.27
Epoch 30, V

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_lr_50-10.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 186.26it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean    55.059847               0.0      0.003196
std     17.736060               0.0      0.001307
min     22.146592               0.0      0.001997
25%     42.015077               0.0      0.002001
50%     54.958444               0.0      0.003000
75%     65.825248               0.0      0.003652
max    109.934351               0.0      0.008518
Number of infeasible solution: 0


## Parametric Learning Then Rounding

In [17]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [18]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 16            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate
batch_size = 64       # batch size

In [19]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# build neuromancer problem for rounding
components = nn.ModuleList([smap]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x"], steepness, num_blocks, penalty_weight)

In [20]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 2272.38
Epoch 1, Validation Loss: 329.52
Epoch 2, Validation Loss: 184.36
Epoch 3, Validation Loss: 82.25
Epoch 4, Validation Loss: 49.15
Epoch 5, Validation Loss: 36.66
Epoch 6, Validation Loss: 32.40
Epoch 7, Validation Loss: 29.87
Epoch 8, Validation Loss: 28.27
Epoch 9, Validation Loss: 26.84
Epoch 10, Validation Loss: 28.04
Epoch 11, Validation Loss: 25.57
Epoch 12, Validation Loss: 24.72
Epoch 13, Validation Loss: 24.81
Epoch 14, Validation Loss: 23.93
Epoch 15, Validation Loss: 24.05
Epoch 16, Validation Loss: 24.39
Epoch 17, Validation Loss: 23.36
Epoch 18, Validation Loss: 24.05
Epoch 19, Validation Loss: 22.74
Epoch 20, Validation Loss: 22.80
Epoch 21, Validation Loss: 24.63
Epoch 22, Validation Loss: 22.72
Epoch 23, Validation Loss: 23.08
Epoch 24, Validation Loss: 24.91
Epoch 25, Validation Loss: 22.42
Epoch 26, Validation Loss: 22.52
Epoch 27, Validation Loss: 28.71
Epoch 28, Validation Loss: 22.87
Epoch 29, Validation Loss: 23.73
Epoch 30, Valida

In [21]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x"]
    for i in range(num_blocks*2):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_pr_50-10.csv")
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 221.02it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean    64.232553          0.391075      0.000988
std     15.186186          0.606805      0.000559
min     34.560756          0.000000      0.000000
25%     53.647430          0.000000      0.000999
50%     62.802589          0.000000      0.001003
75%     73.050173          0.866991      0.001112
max    117.822355          2.986861      0.002522
Number of infeasible solution: 45


### STE Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 16            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundSTEModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
rnd = roundSTEModel(param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], int_ind=model.int_ind, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 2284.40
Epoch 1, Validation Loss: 374.33
Epoch 2, Validation Loss: 219.78
Epoch 3, Validation Loss: 124.98
Epoch 4, Validation Loss: 104.83
Epoch 5, Validation Loss: 95.40
Epoch 6, Validation Loss: 85.53
Epoch 7, Validation Loss: 76.12
Epoch 8, Validation Loss: 71.40
Epoch 9, Validation Loss: 77.14
Epoch 10, Validation Loss: 75.25
Epoch 11, Validation Loss: 71.81
Epoch 12, Validation Loss: 71.41
Epoch 13, Validation Loss: 71.24
Epoch 14, Validation Loss: 69.16
Epoch 15, Validation Loss: 70.23
Epoch 16, Validation Loss: 71.02
Epoch 17, Validation Loss: 68.93
Epoch 18, Validation Loss: 69.23
Epoch 19, Validation Loss: 68.48
Epoch 20, Validation Loss: 68.75
Epoch 21, Validation Loss: 70.44
Epoch 22, Validation Loss: 70.08
Epoch 23, Validation Loss: 71.08
Epoch 24, Validation Loss: 76.17
Epoch 25, Validation Loss: 71.47
Epoch 26, Validation Loss: 71.92
Epoch 27, Validation Loss: 72.65
Epoch 28, Validation Loss: 72.79
Epoch 29, Validation Loss: 72.62
Epoch 30, Vali

In [11]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x"]
    for i in range(num_blocks*2):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_st_50-10.csv")
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 219.05it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean    65.378768          0.000236      0.001611
std     20.891297          0.001685      0.000616
min     35.462500          0.000000      0.000506
25%     50.886689          0.000000      0.001002
50%     63.855934          0.000000      0.001560
75%     74.839186          0.000000      0.002004
max    157.242822          0.013853      0.003570
Number of infeasible solution: 2
